In [5]:
import numpy as np
import pandas as pd
import random
import os
from collections import Counter
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix 
#from sklearn.mixture import BayesianGaussianMixture


In [9]:
#Creating the vocabulary
def create_vocabulary(start,end):
    vocab = []
    overall_length = 0
    for i in range(start, end):
        path_Name = "./datasets/Q10/part"+str(i)
        file_count = ([f for f in os.walk(path_Name).next()[2] if f[-4:] == ".txt"])
        overall_length += len(file_count)
        for files in file_count:
            file_Name = path_Name+'/'+files
            my_File = open(file_Name, 'r')
        #Read the file
            read_File = my_File.read()
        #Split the words
            words = read_File.split()
            words.remove('Subject:')
            for word in words:
                vocab.append(word)
    #Using a set will only save the unique words
    #unique_words = set(words)
    #You can then print the set as a whole or loop through the set etc
    dictionary = Counter(vocab)
    dictionary = dictionary.most_common(3000)
    return dictionary


In [14]:
#Finding the overall length
def measure_length(start,end):
    
    overall_length = 0
    for i in range(start, end):
        path_Name = "./datasets/Q10/part"+str(i)
        file_count = ([f for f in os.walk(path_Name).next()[2] if f[-4:] == ".txt"])
        overall_length += len(file_count)  
    return overall_length


In [15]:
#Now to create the frequency matrix
def create_feature(start, end):
    dictionary = create_vocabulary(start, end)
    overall_length = measure_length(start, end)
    features_matrix = np.zeros((overall_length,len(dictionary)))
    train_labels = np.zeros(overall_length)
    iterate = 0
    for i in range(start, end):
        path_Name = "./datasets/Q10/part"+str(i)
        file_count = ([f for f in os.walk(path_Name).next()[2] if f[-4:] == ".txt"])
        docID = 0;
        for files in file_count:
            file_Name = path_Name+'/'+files
            if(files.find('spmsg')!=-1):
                train_labels[iterate] = 1 
            my_File = open(file_Name, 'r')
        #Read the file
            read_File = my_File.read()
        #Split the words
            words = read_File.split()
            words.remove('Subject:')
            for word in words:
                wordID = 0;
                for i,d in enumerate(dictionary):
                    if d[0] == word:
                    #print "hi"     
                        wordID = i
                        features_matrix[docID,wordID] = words.count(word)
            iterate = iterate + 1
    return features_matrix

In [17]:
train_matrix = create_feature(1, 9)
model1 = MultinomialNB()
model2 = GaussianNB()
model3 = BernoulliNB()

model1.fit(train_matrix,train_labels)
model2.fit(train_matrix,train_labels)
model3.fit(train_matrix,train_labels)

test_matrix = create_feature(9, 11)

result1 = model1.predict(test_matrix)
result2 = model2.predict(test_matrix)
result3 = model3.predict(test_matrix)

print "Multinomial NB"
print confusion_matrix(train_labels,result1)
print model1.score(test_matrix, train_labels)
print "Gaussian NB"
print confusion_matrix(train_labels,result2)
print model2.score(test_matrix, train_labels)
print "Bernoulli NB"
print confusion_matrix(train_labels,result3)
print model3.score(test_matrix, train_labels)


NameError: name 'train_labels' is not defined

NameError: name 'BayesianGaussianMixture' is not defined

In [ ]:
'''#Now to create the frequency matrix
features_matrix = np.zeros((overall_length,400))
for i in range(1, 9):
    path_Name = "./datasets/Q10/part"+str(i)
    file_count = ([f for f in os.walk(path_Name).next()[2] if f[-4:] == ".txt"])
    docID = 0;
    for files in file_count:
        file_Name = path_Name+'/'+files
        my_File = open(file_Name, 'r')
        #Read the file
        read_File = my_File.read()
        #Split the words
        words = read_File.split()
        for word in words:
            wordID = 0;
            for i,d in enumerate(dictionary):
                if d == word:
                #print i, d     
                wordID = i
                features_matrix[docID,wordID] = words.count(word)'''

In [15]:
#Now to create the frequency matrix
features_matrix = np.zeros((overall_length,len(dictionary)))
train_labels = np.zeros(overall_length)
iterate = 0
for i in range(1, 9):
    path_Name = "./datasets/Q10/part"+str(i)
    file_count = ([f for f in os.walk(path_Name).next()[2] if f[-4:] == ".txt"])
    docID = 0;
    for files in file_count:
        file_Name = path_Name+'/'+files
        if(files.find('spmsg')!=-1):
            train_labels[iterate] = 1  #set train labels to 1 if the message is a spam mail     
        my_File = open(file_Name, 'r')
        #Read the file
        read_File = my_File.read()
        #Split the words
        words = read_File.split()
        words.remove('Subject:')
        for word in words:
            wordID = 0;
            for i,d in enumerate(dictionary):
                if d[0] == word:
                    #print "hi"     
                    wordID = i
                    features_matrix[docID,wordID] = words.count(word)
        iterate = iterate + 1

In [17]:
print len(train_labels)

878


In [19]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix 
# Create a dictionary of words with its frequency


#dictionary = make_Dictionary(train_dir)

# Prepare feature vectors per training mail and its labels

#train_labels = np.zeros(overall_length)
#train_labels[351:701] = 1
#train_matrix = extract_features(train_dir)

# Training SVM and Naive bayes classifier
train_matrix = features_matrix
model1 = MultinomialNB()
#model2 = LinearSVC()
model1.fit(train_matrix,train_labels)
#model2.fit(train_matrix,train_labels)

# Test the unseen mails for Spam
#test_dir = 'test-mails'
#test_matrix = extract_features(test_dir)
#test_labels = np.zeros(260)
#test_labels[130:260] = 1
#result1 = model1.predict(test_matrix)
#result2 = model2.predict(test_matrix)
#print confusion_matrix(test_labels,result1)
#print confusion_matrix(test_labels,result2)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)